In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import mysql.connector
import itertools

driver = webdriver.Chrome()
driver.maximize_window()

driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
state = "goa"
time.sleep(3)

wait = WebDriverWait(driver, 5)

def scroll_page(n):
    SCROLL_PAUSE_TIME = n

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_page(5)
bus_route_name_list = []
bus_route_link_list = []

page_number = 1

for i in range(0,4):

    try:
        bus_route_name = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
        for bus in bus_route_name:    
            bus_route_name_list.append(bus.text)
            bus_route_link_list.append(bus.get_attribute("href"))

        pagination_container = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[4]/div[12]')
        next_page_button = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs") and text() = "{int(page_number) + 1}"]')
        
        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        wait.until(EC.element_to_be_clickable(next_page_button))
        next_page_button.click()
        page_number += 1

        time.sleep(2)

    except:
        print("No such page") 
      

No such page


In [6]:
bus_route_name

[<selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa48745cf1c7", element="f.6758FB636CB89D443DE0890D02FEB390.d.251DC9047162BDF35FD2C0712C2BF6B7.e.44")>,
 <selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa48745cf1c7", element="f.6758FB636CB89D443DE0890D02FEB390.d.251DC9047162BDF35FD2C0712C2BF6B7.e.45")>,
 <selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa48745cf1c7", element="f.6758FB636CB89D443DE0890D02FEB390.d.251DC9047162BDF35FD2C0712C2BF6B7.e.46")>,
 <selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa48745cf1c7", element="f.6758FB636CB89D443DE0890D02FEB390.d.251DC9047162BDF35FD2C0712C2BF6B7.e.47")>,
 <selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa48745cf1c7", element="f.6758FB636CB89D443DE0890D02FEB390.d.251DC9047162BDF35FD2C0712C2BF6B7.e.48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="baae673e96654be3557faa487

In [2]:
bus_route_link_list = bus_route_link_list[:2]

In [3]:
state_list = []
bus_route_list = []
bus_r_link_list = []
bus_name_list = []
bustype_list = []  
departing_time_list = []
duration_list = []
reaching_time_list = []
star_rating_list = []
price_list = []
seat_available_list = []

for i,link in enumerate (bus_route_link_list):

    bus_route_index = int(i)
    driver.get(link)

    time.sleep(3)

    wait = WebDriverWait(driver, 5)

    buttons_xpath = '//div[contains(@class, "clearfix")]/div[contains(@class, "w-14")]/div[contains(@class, "button")]'
    buttons = driver.find_elements(By.XPATH, buttons_xpath)

    actions = ActionChains(driver)
    buttons = buttons[0:2]

    for button in buttons:
        actions.move_to_element(button).perform()
        wait.until(EC.element_to_be_clickable(button))
        actions.click(button).perform()
        time.sleep(2)  

        scroll_page(5)

    scroll_page(2)

    elements = driver.find_elements(By.CSS_SELECTOR, '.clearfix.row-one')

    for element in elements:
        state_list.append(state)
        bus_route_list.append(bus_route_name_list[bus_route_index])
        bus_r_link_list.append(link)
        bus_name_list.append(element.find_element(By.CSS_SELECTOR, '.travels').text.strip())
        bustype_list.append(element.find_element(By.CSS_SELECTOR, '.bus-type').text.strip())
        departing_time_list.append(element.find_element(By.CSS_SELECTOR, '.dp-time').text.strip())
        duration_list.append(element.find_element(By.CSS_SELECTOR, '.dur').text.strip())
        reaching_time_list.append(element.find_element(By.CSS_SELECTOR, '.bp-time').text.strip())
        rating_text = element.find_element(By.CSS_SELECTOR, '.column-six.p-right-10.w-10.fl').text
        if rating_text == ' ' or rating_text == 'New':
            star_rating_list.append("0.0")
        else:
            star_rating_list.append(rating_text[:3])
        price_list.append(element.find_element(By.CSS_SELECTOR, '.fare span.f-19.f-bold').text.strip())
        seat_available_list.append(element.find_element(By.CSS_SELECTOR, '.seat-left').text.split()[0].strip())


In [86]:
df = pd.DataFrame({'state':state_list,
                    'route_name':bus_route_list,
                    'route_link':bus_r_link_list,
                    'busname':bus_name_list,
                    'bustype':bustype_list,
                    'departing_time':departing_time_list,
                    'duration':duration_list,
                    'reaching_time':reaching_time_list,
                    'star_rating':star_rating_list,
                    'price':price_list,
                    'seats_available':seat_available_list})

df['departing_time'] = pd.to_datetime(df['departing_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['reaching_time'] = pd.to_datetime(df['reaching_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

df.to_csv("wbtc.csv")

conn = mysql.connector.connect(host = "127.0.0.1",
                               user ="root",
                               password ="Admin@123",
                               database = "red_bus",
                               )

cursor = conn.cursor()

insert_query = """INSERT INTO bus_info_NEW (
            state,
            route_name,
            route_link,
            busname,
            bustype, 
            departing_time, 
            duration,
            reaching_time, 
            star_rating, 
            price,
            seats_available
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s)"""

for index, row in df.iterrows():
    cursor.execute(insert_query, (
        row['state'],
        row['route_name'],
        row['route_link'],
        row['busname'],
        row['bustype'],
        row['departing_time'],
        row['duration'],
        row['reaching_time'],
        row['star_rating'],
        row['price'],
        row['seats_available']
))
conn.commit()        

C:\Users\New\AppData\Local\Temp\ipykernel_25072\4291449401.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['departing_time'] = pd.to_datetime(df['departing_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
C:\Users\New\AppData\Local\Temp\ipykernel_25072\4291449401.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['reaching_time'] = pd.to_datetime(df['reaching_time']).dt.strftime('%Y-%m-%d %H:%M:%S')


In [87]:
state_query  = """select distinct state from bus_info_new;"""
cursor.execute(state_query)
cursor.fetchall()

[('KTCL',),
 ('APSRTC',),
 ('KSRTC (Kerala)',),
 ('TSRTC',),
 ('SBSTC',),
 ('RSRTC',),
 ('HRTC',),
 ('ASTC',),
 ('UPSRTC',),
 ('WBTC',)]